In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
X_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
train.iloc[:, 70:]

,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0)

In [9]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

In [10]:
good_label_cols = [col for col in object_cols if set(X_train[col]) == set(X_valid[col])]

In [11]:
good_label_cols

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'LotConfig',
 'LandSlope',
 'BldgType',
 'HouseStyle',
 'MasVnrType',
 'ExterQual',
 'Foundation',
 'BsmtQual',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'CentralAir',
 'KitchenQual',
 'FireplaceQu',
 'GarageFinish',
 'GarageCond',
 'PavedDrive',
 'Fence',
 'SaleCondition']

In [12]:
bad_label_cols = list(set(object_cols) - set(good_label_cols))

In [19]:
X_train = X_train.drop(bad_label_cols, axis=1)
X_valid = X_valid.drop(bad_label_cols, axis=1)

In [21]:
X_train.describe().T.iloc[:10]

,count,mean,std,min,25%,50%,75%,max
Id,1022.0,731.090998,421.591020,1.0,359.25,745.5,1095.75,1460.0
MSSubClass,1022.0,56.614481,42.160508,20.0,20.00,50.0,70.00,190.0
LotFrontage,833.0,69.668667,23.077851,21.0,58.00,69.0,80.00,313.0
LotArea,1022.0,10567.966732,10291.276619,1300.0,7742.50,9536.0,11596.00,215245.0
OverallQual,1022.0,6.079256,1.357888,1.0,5.00,6.0,7.00,10.0
OverallCond,1022.0,5.562622,1.118578,1.0,5.00,5.0,6.00,9.0
YearBuilt,1022.0,1970.940313,30.242334,1872.0,1954.00,1972.0,2000.00,2010.0
YearRemodAdd,1022.0,1984.698630,20.505826,1950.0,1967.00,1993.0,2003.00,2010.0
MasVnrArea,1017.0,103.553589,181.613707,0.0,0.00,0.0,170.00,1600.0
BsmtFinSF1,1022.0,442.224070,429.925843,0.0,0.00,386.0,707.50,2260.0


In [23]:
X_train.describe(include='object').T.iloc[:10]

,count,unique,top,freq
MSZoning,1022,5,RL,819
Street,1022,2,Pave,1017
Alley,62,2,Pave,32
LotShape,1022,4,Reg,642
LandContour,1022,4,Lvl,923
LotConfig,1022,5,Inside,741
LandSlope,1022,3,Gtl,962
BldgType,1022,5,1Fam,858
HouseStyle,1022,8,1Story,508
MasVnrType,408,3,BrkFace,301


In [24]:
above_0_missing = X_train.isnull().sum() > 0

In [25]:
X_train.isnull().sum()[above_0_missing]

LotFrontage     189
Alley           960
MasVnrType      614
MasVnrArea        5
BsmtQual         24
BsmtExposure     24
BsmtFinType1     24
BsmtFinType2     25
FireplaceQu     478
GarageYrBlt      54
GarageFinish     54
GarageCond       54
Fence           831
dtype: int64

In [26]:
numerical_features = X_train.select_dtypes(include='number').columns.tolist()

In [29]:
len(numerical_features)

37

In [28]:
print(numerical_features)

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


In [30]:
categorical_features = X_train.select_dtypes(exclude='number').columns.tolist()

In [31]:
len(categorical_features)

24

In [32]:
print(categorical_features)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'BldgType', 'HouseStyle', 'MasVnrType', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageFinish', 'GarageCond', 'PavedDrive', 'Fence', 'SaleCondition']


In [33]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

In [34]:
numeric_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [35]:
numeric_pipeline.fit_transform(X_train.select_dtypes(include='number'))

array([[0.04386566, 0.23529412, 0.16667352, ..., 0.        , 0.09090909,
        0.75      ],
       [0.46744345, 0.58823529, 0.16667352, ..., 0.        , 0.90909091,
        0.5       ],
       [0.65798492, 0.        , 0.09931507, ..., 0.        , 0.09090909,
        1.        ],
       ...,
       [0.83344757, 0.41176471, 0.1609589 , ..., 0.        , 0.27272727,
        1.        ],
       [0.38313914, 0.58823529, 0.16667352, ..., 0.        , 0.81818182,
        0.        ],
       [0.46881426, 0.23529412, 0.12671233, ..., 0.        , 0.45454545,
        1.        ]])

In [36]:
from sklearn.compose import ColumnTransformer

In [37]:
full_processor = ColumnTransformer(transformers=[
    ('number', numeric_pipeline, numerical_features),
    ('object', categorical_pipeline, categorical_features)
])

In [38]:
full_processor.fit_transform(X_train)

array([[0.04386566, 0.23529412, 0.16667352, ..., 0.        , 1.        ,
        0.        ],
       [0.46744345, 0.58823529, 0.16667352, ..., 0.        , 1.        ,
        0.        ],
       [0.65798492, 0.        , 0.09931507, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.83344757, 0.41176471, 0.1609589 , ..., 0.        , 1.        ,
        0.        ],
       [0.38313914, 0.58823529, 0.16667352, ..., 0.        , 1.        ,
        0.        ],
       [0.46881426, 0.23529412, 0.12671233, ..., 0.        , 1.        ,
        0.        ]])

In [39]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

In [40]:
lasso = Lasso(alpha=0.1)

In [41]:
lasso_pipeline = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', lasso)
])

In [42]:
_ = lasso_pipeline.fit(X_train, y_train)

In [43]:
preds = lasso_pipeline.predict(X_valid)

In [44]:
mean_absolute_error(y_valid, preds)

19757.858999389497

In [45]:
lasso_pipeline.score(X_valid, y_valid)

0.7624053679096316

In [46]:
from sklearn.model_selection import GridSearchCV

In [47]:
param_dict = {'model__alpha': np.arange(0, 1, 0.05)}

In [48]:
search = GridSearchCV(lasso_pipeline, param_dict, 
                      cv=10, 
                      scoring='neg_mean_absolute_error')

_ = search.fit(X_train, y_train)

In [49]:
print('Best score:', abs(search.best_score_))

Best score: 18878.047812197863


In [50]:
print('Best alpha:', search.best_params_)

Best alpha: {'model__alpha': 0.9500000000000001}


In [51]:
param_dict = {'model__alpha': np.arange(1, 200, 5)}

search = GridSearchCV(lasso_pipeline, param_dict, 
                      cv=10, 
                      scoring='neg_mean_absolute_error')

_ = search.fit(X_train, y_train)

In [52]:
print('Best score:', abs(search.best_score_))

Best score: 17969.963207467303


In [53]:
print('Best alpha:', search.best_params_)

Best alpha: {'model__alpha': 136}


In [54]:
lasso = Lasso(alpha=136)

final_lasso_pipe = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', lasso)
])

In [55]:
_ = final_lasso_pipe.fit(X_train, y_train)
preds = final_lasso_pipe.predict(X_valid)

mean_absolute_error(y_valid, preds)

19185.10811456456